In [88]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.stats import chi2_contingency

import plotly.express as px
import plotly.graph_objects as go

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [45]:
path_query_result = '/Users/s.kostyuchik/projects/jupyter_working/story_mob_analytisc/query_result_2022-07-13T08.csv'
path_isc = '/Users/s.kostyuchik/projects/jupyter_working/story_mob_analytisc/story_mob_final.csv'
story_info = pd.read_csv(path_query_result)
data = pd.read_csv(path_isc)

In [152]:
d1 = pd.read_csv('interfaces_story_clicks.csv')

In [154]:
d1.person_party_rk.nunique()

72767

In [46]:
story_info

,name,storyId,mindate,maxdate,days,uimp,uc,open_rate,cookie,cookie_rate,button,button_prc,like,like_prc,dis,dis_prc,page_1,page_2_prc,page_3_prc,page_4_prc,page_5_prc,page_6_prc
0,SME::Предложения банка. Как улучшили приложени...,16082,2022-03-22 15:17:25,2022-07-01 00:45:30,100,240170,27744,"11,55181746",20865,"75,20544983",238,"0,8578431373",129,"0,4649653979",14,"0,05046136101",27066,"60,82908446","89,14601555","92,44395994","92,77712264","90,12551636"
1,SME::Предложения банка. Как улучшили приложени...,16081,2022-03-22 15:16:48,2022-06-30 23:55:07,100,262940,29115,"11,07286833",20123,"69,11557616",246,"0,8449252962",152,"0,5220676627",12,"0,04121586811",29153,"72,45909512","90,7971975","92,56517205","91,98490481","90,06796889"
2,SME::Предложения банка. Как улучшили приложени...,16279,2022-03-22 15:18:43,2022-05-16 23:44:56,55,119960,5484,"4,571523841",3708,"67,61487965",84,"1,531728665",30,"0,5470459519",3,"0,05470459519",5541,"74,53528244","91,38014528","92,66030737","92,42207607","90,81064356"
3,SME::Предложения банка. Как улучшили приложени...,16280,2022-03-22 15:18:05,2022-05-16 23:44:35,55,110311,5276,"4,78284124",3244,"61,48597422",62,"1,175132676",27,"0,5117513268",4,"0,07581501137",5139,"63,10566258","90,7493062","93,57798165","91,902687","90,20150138"


In [136]:
d2 = data.copy()

In [143]:
d2['period'] = d2['period'].apply(lambda x: pd.Timestamp(x).to_period('M').to_timestamp().date())

In [146]:
d2 = d2.groupby('period').sum().drop('session_share', axis=1).reset_index()

In [148]:
d2['session_share'] = d2['session_mob'] / (d2['session_mob'] + d2['session_web'])

In [150]:
fig = px.line(d2, x='period', y='session_share')
fig.show()

In [101]:
#data.session_share = data.session_share.apply(lambda x: round(x, 2))
data.period = pd.to_datetime(data.period)

In [106]:
fig = px.scatter(data, x='period', y='session_share',
                 template = 'plotly_dark',
                 trendline = 'ols',
                 title = "Доля сессий по пользователям, зарегистрировавшим бизнес 2 и более года назад")

fig.update_traces(mode = 'lines')
fig.data[-1].line.color = 'red'

fig.add_vline(x='2022-03-22', line_width=3, line_dash="dash", line_color="green")

fig.add_annotation(x='2022-03-22', y=0.934,
            text="Показали историю",
            showarrow=True,
            arrowhead=1)

fig.add_trace(go.Scatter(x=data.period,
                         y=data.session_share,
                         text=[f'M {r.person_mob}<br>W {r.person_web}' for r in data.itertuples()],
                         mode="text",
                         textposition="top center",
                         showlegend=False))

fig.add_annotation(text='''Доля сессий на мобиле в общей доле сессий (моб+веб)<br>M - количество пользователей на мобилке, W - количество пользователей на вебе<br>Красным изображена линия тренда''', 
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.18,
                    y=0.02,
                    bordercolor='black',
                    borderwidth=1)


fig.show()

In [73]:
data['p'] = data.period.apply(lambda x: 'before' if x < '2022-03-22' else 'after')

In [77]:
d_agg = data.groupby('p')[['session_mob', 'session_web']].sum()

In [92]:
d_agg['share_mob'] = d_agg['session_mob'] / (d_agg['session_mob'] + d_agg['session_web'])

In [86]:
a_s = d_agg[d_agg.index=='before']['session_mob'].squeeze()
a_f = d_agg[d_agg.index=='before']['session_web'].squeeze()
b_s = d_agg[d_agg.index=='after']['session_mob'].squeeze()
b_f = d_agg[d_agg.index=='after']['session_web'].squeeze()

successes, fails = [a_s, b_s], [a_f, b_f]

In [105]:
d_agg

,session_mob,session_web,share_mob
p,,,
after,5764326,436222,0.929648
before,3540757,293391,0.923479


In [94]:
print(f'P-val: {chi2_contingency(np.array([fails, successes]))[1]}')

P-val: 1.0825930379541144e-292


In [ ]:
#погрешность в данных
#общее направление тренда